In [1]:
import csv
import numpy as np
import random as rnd
import copy as cp

# Market chain class

In [2]:
class Market_MC:
    def __init__(self, filepath, gap = 0, end = float("inf"), col = 1):
        self.filepath = filepath
        values = []
        #Read the market daily prices form given file
        with open(filepath) as csvData:
            csvReader = csv.reader(csvData)
            i = 0
            for row in csvReader:
                if i >= gap and i <= end:
                    values.append(float(row[col]))
                i += 1
        
        self.value = np.array(values)
        
    def update_params(self, period, start = 0):
    
        movement = self.value[start + 1:(start + period-1)] >= self.value[start :(start + period-2)]
        #Compute transition probabilities
        b_t_b = np.sum(~(movement[1:] | movement[0:-1]))/(period - 2)
        g_t_g = np.sum((movement[1:] & movement[0:-1]))/(period - 2)
        self.probs = np.array([b_t_b,1-b_t_b,1-g_t_g,g_t_g]).reshape((2,2))
        #Update market state
        self.state = 1 if movement[-1] else 0
        
        return movement
        
    def simulate_market(self, length):
        #Sample lenght random numbers in (0,1)
        rnd_vals = [rnd.uniform(0,1) for i in range(length)]
        new_state = 1
        if rnd_vals[0] < self.probs[self.state][0]:
            new_state = 0
        market_states = [new_state]
    
        for idx, x in enumerate(rnd_vals[1:]):
            #update market state
            new_state = 1
            if x < self.probs[market_states[idx]][0]:
                new_state = 0
            market_states.append(new_state)
        
        return np.array(market_states)

    def set_state(self, state):
        self.state = state

# Stock chain class

In [200]:
class Stock_MC:
    
    def __init__(self, filepath, gap = 0, end = float("inf"), col = 1):
        self.filepath = filepath
        values = []
        #Read the stock daily prices form given file
        with open(filepath) as csvData:
            csvReader = csv.reader(csvData)
            i = 0
            for row in csvReader:
                if i >= gap and i <= end:
                    values.append(float(row[col]))
                i += 1
        
        self.value = np.array(values)
        
    def update_params(self, market_movement, period, start = 0, delta_type = 0):
    
        
        ups = self.value[start + 1:(start + period-1)] >= self.value[start :(start + period-2)]
        #Compute transition probabilities
        
        #Probabilities for bad market 
        up_b = np.sum(ups[1:] & ~market_movement[1:])/np.sum(~market_movement[1:])
        #self.probs[0][1] = np.sum(ups[1:] & ~market_movement[1:])/(period - 1)
        #self.probs[0][0] = 1 - self.probs[0][1]
        #Probabilities for good market
        up_g = np.sum(ups[1:] & market_movement[1:])/np.sum(market_movement[1:])
        #self.probs[1][1] = np.sum(ups[1:] & market_movement[0:-1])/(period - 1)
        #self.probs[1][0] = 1 - self.probs[1][1]
        self.probs = np.array([1-up_b, up_b, 1-up_g, up_g]).reshape((2,2))
        #Update stock state
        self.state = self.value[start + period - 1]
        #Update stock delta parameter
        if delta_type == 0:
                #Use mean variation as delta
                self.delta = np.mean(np.absolute(self.value[start + 1:(start + period-1)] - self.value[start :(start + period-2)]))
        elif delta_type == 1:
                #Use median variation as delta
                self.delta = np.median(np.absolute(self.value[start + 1:(start + period-1)] - self.value[start :(start + period-2)]))
        elif delta_type == 2:
                #Use minimum variation as delta
                self.delta = np.quantile(np.absolute(self.value[start + 1:(start + period-1)] - self.value[start :(start + period-2)]),0.1)
        
        
    def simulate_stock_trajectory(self, market_state, length):
        #Sample lenght random numbers in (0,1)
        rnd_vals = [rnd.uniform(0,1) for i in range(length)]
        stock_states = [(self.state - self.delta) if (rnd_vals[0] <= self.probs[market_state[0]][0]) else (self.state + self.delta)]
            
        for idx, x in enumerate(rnd_vals[1:]):
            #update stock state
            if x <= self.probs[market_state[idx+1]][0]:
                stock_states.append(stock_states[idx] - self.delta)
            else:                           
                stock_states.append(stock_states[idx] + self.delta)

            
        return np.array(stock_states)
    
    def simulate_stock_price(self, market_state, length):
        #Sample lenght random numbers in (0,1)
        rnd_vals = [rnd.uniform(0,1) for i in range(length)]
        end_value = cp.copy(self.state)
        for idx, x in enumerate(rnd_vals):
            #update stock state
            if x <= self.probs[market_state[idx]][0]:
                end_value -= self.delta
            else:
                end_value += self.delta
                
        return end_value
        
    def set_state(self, state):
        self.state = state

In [151]:
market = Market_MC('./data/market_prices.csv')
move = market.update_params(500)
market_simul = market.simulate_market(30)

In [263]:
stock1 = Stock_MC('./data/AAPL_prices.csv')
stock1.update_params(move, 500,delta_type=0)

In [264]:
print("probs")
print(stock1.probs)
print("delta")
print(stock1.delta)
print("state")
print(stock1.state)
val = 0
for x in range(1000):
    val += stock1.simulate_stock_price(market_simul, 30)
val/1000

probs
[[0.58490566 0.41509434]
 [0.39298246 0.60701754]]
delta
0.8891967871485945
state
118.65


119.71347935742999